In [3]:
import pandas as pd
import plotly.express as px

In [16]:
# ──────────────────────────────────────────────────────────────────────────
# 1) CARREGAR O JSON CLASSIFICADO
# ──────────────────────────────────────────────────────────────────────────
df = pd.read_json("comentarios_NegoBorel_Classificados.json", encoding="utf-8")

In [14]:
# ──────────────────────────────────────────────────────────────────────────
# 2) PREPARAR COLUNA "tempoPlot" EM DATETIME (HH:MM)
# ──────────────────────────────────────────────────────────────────────────
if "publishedAt" in df.columns:
    # Caminho 1 – data/hora real ------------------------------
    df["tempoPlot"] = pd.to_datetime(df["publishedAt"]).dt.floor("1min")

elif "time_elapsed" in df.columns:
    # Caminho 2 – duração HH:MM:SS ----------------------------
    # 1. Converte para Timedelta
    df["tempo_dur"] = pd.to_timedelta(df["time_elapsed"])
    # 2. Soma a um dia base (1970-01-01) → vira datetime
    base = pd.Timestamp("1970-01-01")
    df["tempoPlot"] = (base + df["tempo_dur"]).dt.floor("1min")

else:
    raise ValueError("Coluna de tempo não encontrada ('timestamp' ou 'time_elapsed').")

# ──────────────────────────────────────────────────────────────────────────
# 3) BINARIZAR MÉTRICAS (score > 0.5)
# ──────────────────────────────────────────────────────────────────────────
rótulos = [
    "toxicity", "severe_toxicity", "obscene", "threat",
    "insult", "identity_attack", "sexual_explicit"
]
for r in rótulos:
    df[f"{r}_bin"] = df[r] > 0.5

# ──────────────────────────────────────────────────────────────────────────
# 4) CONTAR COMENTÁRIOS POR MINUTO
# ──────────────────────────────────────────────────────────────────────────
dfAgg = (
    df.groupby("tempoPlot")[[f"{r}_bin" for r in rótulos]]
      .sum()
      .reset_index()
)

# ──────────────────────────────────────────────────────────────────────────
# 5) TRANSFORMAR PARA FORMATO LONGO E PLOTAR
# ──────────────────────────────────────────────────────────────────────────
dfLongo = (
    dfAgg
      .melt(id_vars="tempoPlot", var_name="tipo", value_name="quantidade")
      .assign(tipo=lambda d: d["tipo"].str.replace("_bin", ""))
)

fig = px.line(
    dfLongo,
    x="tempoPlot",
    y="quantidade",
    color="tipo",
    markers=True,
    title="Comentários Monark por Tipo de Violência (HH:MM)",
    labels={"tempoPlot": "Hora e minuto", "quantidade": "Nº de Comentários"}
)

# formata eixo X para mostrar só HH:MM
fig.update_xaxes(tickformat="%H:%M", title_text="Hora (HH:MM)")
fig.update_layout(hovermode="x unified")
fig.show()


In [17]:
# Selecionar apenas colunas de discurso de ódio
hate_columns = ['toxicity', 'severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat', 'sexual_explicit']

# Filtrar apenas os comentários com probabilidade superior a 0.7
filtered_df = df[(df[hate_columns] > 0.7).any(axis=1)]

# Contar o total de cada tipo de discurso de ódio
hate_counts = (filtered_df[hate_columns] > 0.7).sum()

# Criar o gráfico de barras
fig = px.bar(
    hate_counts,
    x=hate_counts.index,
    y=hate_counts.values,
    labels={'x': 'Tipo de Discurso de Ódio', 'y': 'Total de Comentários'},
    title='Caso Nego do Borel: Total de Comentários com Alto Índice de Discurso de Ódio (Probabilidade > 0.7)'
)

# Exibir o gráfico
fig.show()
